In [4]:
import collections
import math
from typing import Sequence

import geopandas as gpd
import laspy
import numpy as np
import startinpy

import raster
from feature.utils import timing

In [64]:
from __future__ import annotations


class PointCloud:
    @timing
    def __init__(self, filename: str) -> None:
        with laspy.open(filename) as f:
            self.las = f.read()

    @timing
    def crop(self, bbox: Sequence[float]) -> PointCloud:
        min_x = (bbox[0] - self.las.header.x_offset) / self.las.header.x_scale
        min_y = (bbox[1] - self.las.header.y_offset) / self.las.header.y_scale
        max_x = (bbox[2] - self.las.header.x_offset) / self.las.header.x_scale
        max_y = (bbox[3] - self.las.header.y_offset) / self.las.header.y_scale
        self.las.points = self.las.points[np.logical_and(np.logical_and(min_x <= self.las.X, self.las.X <= max_x),
                                                         np.logical_and(min_y <= self.las.Y, self.las.Y <= max_y))]
        return self

    def save(self, filename: str) -> None:
        with laspy.open(filename, "w", header=self.las.header, do_compress=True) as f:
            f.write_points(self.las.points)

    def __len__(self) -> int:
        return len(self.las.points)

    def __getitem__(self, key: int):
        ...


# FIXME: Rewrite this function as a PointCloud method.
@timing
def to_gdf(pc: PointCloud) -> gpd.GeoDataFrame:
    return gpd.GeoDataFrame({"id": np.arange(len(pc)), "geometry": gpd.points_from_xy(pc.las.x, pc.las.y)},
                            crs="EPSG:28992")


# FIXME: Rewrite this function as a PointCloud method.
@timing
def intersect(pc: PointCloud, objs: gpd.GeoDataFrame) -> np.ndarray:
    return to_gdf(pc).overlay(objs)["id_1"].to_numpy()

In [69]:
pc = PointCloud("C:/Documents/RoofSense/data/temp/37EN2_16.LAZ")
print(pc.las.points)
pc.crop([84489.9256187045539264, 445795.0548930063378066, 85685.1996997416717932, 446993.4567845478304662]).save(
    '37EN2_16.LAZ_crop.laz')


func __init__ :-> 3154 ms
<ScaleAwarePointRecord(fmt: <PointFormat(8, 6 bytes of extra dims)>, len: 50334569, point size: 44)>
func crop :-> 1429 ms


In [3]:
# class BufferableGeoDataFrame(gpd.GeoDataFrame):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__()
# 
#     def buffer(self, *args, **kwargs) -> None:
#         ...

@timing
def __init__(filename: str) -> gpd.GeoDataFrame:
    return gpd.read_file(filename)


@timing
# TODO: Review the various buffer options.
# TODO: Rewrite this function as an appropriate class method.
def buffer(objs: gpd.GeoDataFrame, dist: float = 10) -> None:
    objs["geometry"] = objs["geometry"].buffer(dist)

## Step 1
Load the point cloud into memory.

In [4]:
pc = PointCloud("../aula.LAZ")

func __init__ :-> 289 ms


## Step 2
Load the building footprints into memory.

In [5]:
fps = __init__("C:/Documents/RoofSense/data/temp/9-284-556.buildings.gpkg")

func __init__ :-> 774 ms


## Step 3
Buffer the footprints by 10 m.

NOTE: The buffer distance is selected such that most if not all the natural neighbors of 
      the points along the footprint edges are included in the subsequent steps.

In [6]:
buffer(fps)

func buffer :-> 74 ms


## Step 4
Intersect the point cloud with the buffers.

In [7]:
ids = intersect(pc, fps)

func to_gdf :-> 2318 ms
func intersect :-> 32190 ms


Filter the point cloud and detach the point records which are required in the subsequent
steps.

In [8]:
# TOSELF: Overwrite the original point cloud?
pts = pc.las.points[ids]
pts = np.vstack((pts.x, pts.y,  # NOTE: The point elevation must be used instead of e.g., reflectance
                 #       because it is required in the subsequent steps.
                 pts.z)).transpose()

## Step 5
Compute the Delaunay triangulation of the remaining points.

In [9]:
# FIXME: Rewrite this block as a PointCloud method and improve its documentation.

dt = startinpy.DT()
# NOTE: The snap tolerance cannot be set to zero (i.e., disabled) so the nearest 
#       distinct value is used instead.
dt.snap_tolerance = math.ulp(0)

# Maintain a reference to the duplicate PC vertices.
# NOTE: Two or more vertices are considered to be duplicate when their two-dimensional 
#       projections on the Euclidean plane are identical. However. they can obviously 
#       differ in n-dimensional space. In this context these vertices are rechecked 
#       after the DT has been constructed so that the one with the highest elevation is 
#       actually inserted.
tentative_pts: dict[int,  # NOTE: There may be more than one duplicate points.
list[int]] = collections.defaultdict(list)
# Maintain a reference to the finalized PC vertices.
finalized_pts: dict[int, int] = {}

candidate_id: int  # The ID of a candidate vertex in the PC.
tentative_id: int  # The ID of a candidate vertex in the DT.
finalized_id: int  # The ID of a candidate vertex in the DT.

tentative_id = 1
for candidate_id, pt in enumerate(pts):
    finalized_id = dt.insert_one_pt(*pt)
    if finalized_id == tentative_id:
        finalized_pts[finalized_id] = candidate_id
        tentative_id += 1
    else:
        tentative_pts[finalized_id].append(candidate_id)

# NOTE: This array is compiled on demand.
dt_pts = dt.points
for finalized_id, candidate_ids in tentative_pts.items():
    for candidate_id in candidate_ids:
        if dt_pts[finalized_id][2] > pts[candidate_id][2]:
            dt.remove(finalized_id)
            dt.insert_one_pt(*pts[candidate_id])
            # Replace the previous ID of the vertex in the PC with the current one.
            finalized_pts[finalized_id] = candidate_id

Confirm that the resulting lookup table is correct.

In [10]:
np.allclose(dt.points[1:], pts[list(finalized_pts.values())])

True

## Step 6
 Estimate the slope field of the corresponding network.


TODO

In [11]:
# TOSELF: Promote this argument to an environment variable?
CELL_SIZE = 0.25

grid = raster.Raster(CELL_SIZE, dt.get_bbox())

# FIXME: Integrate this block into the Raster initializer.
# Construct the grid.
# TODO: Add documentation.
rows, cols = np.mgrid[grid.len_y - 1:-1:-1, 0:grid.len_x]
# TODO: Add documentation.
xx = grid.bbox[0] + CELL_SIZE * (cols + 0.5)
yy = grid.bbox[1] + CELL_SIZE * (rows + 0.5)
# TODO: Add documentation.
cells = np.column_stack([xx.ravel(), yy.ravel()])

TODO

In [12]:
# TODO: Speed up this block.
# TOSELF: Compute the slope for all faces to avoid additional calls to dt.locate?
valid_cells = []
valid_faces = []
for i, center in enumerate(cells):
    try:
        valid_faces.append(dt.locate(*center))
    except Exception:
        continue
    valid_cells.append(i)

# TOSELF: Discard duplicate faces?
# valid_faces = np.unique(valid_faces, axis=0)

In [13]:
# TODO: Add documentation.

valid_faces = np.array(valid_faces)
v1 = dt_pts[valid_faces[:, 0]]
v2 = dt_pts[valid_faces[:, 1]]
v3 = dt_pts[valid_faces[:, 2]]

u = v2 - v1
v = v3 - v1

n = np.cross(u, v)
n = n / np.linalg.norm(n, axis=1)[:, None]

z = np.array([0, 0, 1])
s = np.degrees(np.arccos(np.dot(n, z)))

In [14]:
grid._Raster__data[np.divmod(valid_cells, grid.len_x)] = s
grid.save("../aula_slpe.tiff")

## Step 6
 Rasterize the slope and reflectance fields of the corresponding network.

In [15]:
# Interpolate the field values at the cell centers.
vals = dt.interpolate({"method": "Laplace"}, cells)

# Populate the raster.
grid._Raster__data = vals.reshape((grid.len_y, grid.len_x))

# Save the raster.
grid.save("../aula_elev.tiff")